In [3]:
from sympy import *
import operator
import functools
init_printing()

In [137]:
import numpy as np
from itertools import combinations

class _tree_parser__(object):
    def __init__(self, in_degree_list):
        self.size = len(in_degree_list)
        self.__in_degree_list = np.array(in_degree_list)
        self.__parents_list = np.zeros(self.size, np.int)
        self.__subtree_size = np.zeros(self.size, np.int)
        self._visitor = -1
        self.__init_sub_trees__()
    
    @property
    def parents(self):  return self.__parents_list
    @property
    def sub_tree_sizes(self):  return self.__subtree_size
    @property
    def leaf_nodes(self):  return np.where(self.__subtree_size==1)[0]
    @property 
    def edges(self):return list(range(1,self.size))
    
    def cut(self, cut_set):
        """
        return a binary parition of nodes in or not in
        """
        v = np.ones(self.size, np.int)
        for c in cut_set: self.walk_down_from(c,v)
        return v
    
    def __is_cut_admissable__(self, cut_set):
        for l in self.leaf_nodes:
            counter = 0
            if l in cut_set:counter+=1
            parent = self.__parents_list[l]
            while parent != -1:#walk to root
                #if any path we pass more than 1 edges in the cut set returned false
                if parent in cut_set:counter+=1
                if counter > 1:return false
                parent = self.__parents_list[parent]
        return True
            
    @property
    def admissable_cuts(self):return self.__admissable_cuts__()
    
    def __admissable_cuts__(self, order=None, complement=False):
        #by default consider all cuts below the root
        if order == None: order = self.size-1
        for o in range(1,order+1):
            for p in combinations(tp.edges,r=o):
                if self.__is_cut_admissable__(list(p)) != complement: 
                    yield p
    
    def walk_down_from(self, i, v):
        v[i] = 0
        for child in np.where(tp.parents==i)[0]: self.walk_down_from(child,v)
            
    def __init_sub_trees__(self,pid=-1):
        self._visitor += 1
        self.__parents_list[self._visitor] = pid
        parent = self._visitor
        for i in range(self.__in_degree_list[self._visitor]): 
            self.__init_sub_trees__(parent)
        #however many nodes i visisted add to sub_tree(root) size
        self.__subtree_size[parent] = self._visitor - parent + 1
        
tp = _tree_parser__([3,2,0,2,0,0,1,0,0])     

In [12]:
#define all the operations and document the Hopf algebra of rooted trees
#most rendering of expressions can be in latex with special display for trees - have unitrees go straight down and simblings branch

#if in general a rooted tree is the set of bin values that is a forest then multiplication will make sense
#graft then can take a set of arguments but probably only support binary

class rooted_tree(object):
    def __init__(self,bin_value=0):#1 must be corrected to 0 for a valid binary tree? e.g. 100 has three nodes
        self._value=bin_value

    def _display_(self):    
        pass
    
    #tree walkers    
        
    def __repr__(self):return self._repr_latex_()
    
    def __mul__(self,a):
        #return rooted tree of the set
        return [a,  self.value]
    
    @property
    def value(self):
        return Symbol("\\bullet_{\{ "+str(self._value)+" \}}",commutative=False)
    
    def __call__(self):
        return self.value
    
    def _sympy_(self):return self.__call()
        
    def html(self):
        d ="""<circle cx="5" cy="5" r="2" stroke="black" stroke-width="1" fill="black" /> """
        svg =  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="10" height="10">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(d, 0,0)
        return "<table style=\"background-color:white\"> <tr> <td> </td><td>"+svg+"</td><td>$$\otimes$$</td></tr></table>"
    
    def _repr_html_(self): 
        return self._repr_latex_()
        return self.html()
    
    def _repr_latex_(self):  return latex(self.value,mode='inline')
    
    #latex version renders self as a symbol(bin)?s

In [13]:
rooted_tree(5)

$\bullet_{\{ 5 \}}$

In [14]:
Symbol("\mathbb{1}"), Function("Delta")(1),Function("B_+")(1,1),Function("m")(1,2)

In [16]:
from sympy.physics.quantum import TensorProduct

A= TensorProduct(Symbol("\mathbb{1}",commutative=False), Symbol("\mathbb{1}",commutative=False))

#B = TensorProduct(rooted_tree(5), rooted_tree(7))
rooted_tree(5) * rooted_tree(5)

[$\bullet_{\{ 5 \}}$, \bullet_{\{ 5 \}}]

In [17]:
#SOME EXAMPLES OF SYB CLASSES THAT SEEMED TO WORK

# A = Symbol('A',commutative=True)

# B = Symbol('B',commutative=True)
# C = Symbol('C',commutative=False)
# D = Symbol('D',commutative=False)

# V = TensorProduct(A,B) + TensorProduct(C,D)
# V

# import sympy as sp
# from sympy.core.expr import Expr

# class Tensor(Expr):
#     __slots__ = ['is_commutative']

#     def __new__(cls, l, r):
#         l = sp.sympify(l)
#         r = sp.sympify(r)
#         obj = Expr.__new__(cls, l, r)
#         obj.is_commutative = False
#         return obj
    
#     def _print(self, e=None):return self._display_()
#     def __srepr__(self):  return self._display_()
#     def __repr__(self):  return self._display_()
    
#     def _display_(self):    
#         body ="""<circle cx="20" cy="20" r="3" stroke="black" stroke-width="1" fill="black" /> """
#         return  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="500" height="120">
#       <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(body, 20,20)
#         #return latex("bullet",mode='inline')

        
#     def _repr_latex_(self):    
#         body ="""<circle cx="20" cy="20" r="3" stroke="black" stroke-width="1" fill="black" /> """
#         return  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="500" height="120">
#       <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(body, 20,20)
#         #return latex("bullet",mode='inline')

#     def __neg__(self):
#         return self

#     def __mul__(self, other):
#         if isinstance(other, Tensor):
#             return Tensor(self.args[0] * other.args[0], other.args[1] * self.args[1])
#         elif other.is_number:
#             if other % 2 == 0:
#                 return 0
#             else:
#                 return self
#         else:
#             return sp.Mul(self, other)


# x, y = sp.symbols('x, y', commutative=False)

# Ym = Tensor(y, 1) - Tensor(1, y)
# Yp = Tensor(y, 1) + Tensor(1, y)
# Xm = Tensor(x, 1) - Tensor(1, x)

# d1 = Ym * Yp + Xm * 0
# # print(d1)
# # print(sp.expand(d1))

# # d2 = Xm * Ym
# # print(d2)
# # print(sp.expand(d2))
# #TensorProduct(Tensor(y, 1),Tensor(y, 1))